In [214]:
CLIENT_ID = "doZap8T-60t8-8SsgzKPxg"
# Add secret key
SECRET_KEY  = "MTfNpx9VzIj1ZB6vk6NSDnYeEtpjxA"

In [215]:
with open('pwd.txt', 'r') as file:
    pw = file.read()

# Getting Access To Reddit API

In [216]:
import praw
reddit = praw.Reddit(client_id = CLIENT_ID,
                     client_secret = SECRET_KEY,
                     username = 'Carlosshadow5',
                     password = pw,
                     user_agent = 'NPL_Project')

# Getting Data by Search

In [217]:
import pandas as pd

# Gets the subreddit
subreddit = reddit.subreddit("BigFive")

## Neuroticism Posts

In [218]:
neuroticism_data = {}
neuroticism_df = pd.DataFrame(neuroticism_data)
neuroticism_user_names = []

subreddit_search = subreddit.search("neuroticism", limit = None)
for post in subreddit_search:
    neuroticism_user_names.append(post.author.name)

# Get unique only
neuroticism_user_names = list(set(neuroticism_user_names))

neuroticism_df['User Name'] = neuroticism_user_names
neuroticism_df

,User Name
0,Zestyclose-Tower3859
1,Proud_Echidna_4960
2,Aegeblomme_MinouKane
3,Reghretti
4,RoseTheQuartz37
...,...
169,lR5Yl
170,Cognitively_Absurd
171,_un1ty
172,killerlumberjack


## Extraversion Posts

In [219]:
extraversion_data = {}
extraversion_df = pd.DataFrame(extraversion_data)
extraversion_user_names = []

subreddit_search = subreddit.search("extraversion", limit = None)
for post in subreddit_search:
    extraversion_user_names.append(post.author.name)

# Get unique only
extraversion_user_names = list(set(extraversion_user_names))

extraversion_df['User Name'] = extraversion_user_names
extraversion_df

,User Name
0,Proud_Echidna_4960
1,raisinindasun
2,Logical_Row3506
3,Aegeblomme_MinouKane
4,abbyrosalina
...,...
119,PalpitationSlight751
120,Cognitively_Absurd
121,_un1ty
122,killerlumberjack


## Openness Posts

In [220]:
openness_data = {}
openness_df = pd.DataFrame(openness_data)
openness_user_names = []

subreddit_search = subreddit.search("openness", limit = None)
for post in subreddit_search:
    openness_user_names.append(post.author.name)

# Get unique only
openness_user_names = list(set(openness_user_names)) 

openness_df['User Name'] = openness_user_names
openness_df

,User Name
0,Proud_Echidna_4960
1,raisinindasun
2,American-_-Nightmare
3,Aegeblomme_MinouKane
4,Reghretti
...,...
172,PalpitationSlight751
173,Cognitively_Absurd
174,_un1ty
175,killerlumberjack


## Agreeableness Posts

In [221]:
agreeableness_data = {}
agreeableness_df = pd.DataFrame(agreeableness_data)
agreeableness_user_names = []

subreddit_search = subreddit.search("agreeableness", limit = None)
for post in subreddit_search:
    agreeableness_user_names.append(post.author.name)

# Get unique only
agreeableness_user_names = list(set(agreeableness_user_names)) 

agreeableness_df['User Name'] = agreeableness_user_names
agreeableness_df

,User Name
0,Zestyclose-Tower3859
1,nationofcake
2,Proud_Echidna_4960
3,Logical_Row3506
4,Aegeblomme_MinouKane
...,...
166,Cognitively_Absurd
167,luckis4losersz
168,_un1ty
169,killerlumberjack


## Conscientiousness Posts

In [222]:
conscientiousness_data = {}
conscientiousness_df = pd.DataFrame(conscientiousness_data)
conscientiousness_user_names = []

subreddit_search = subreddit.search("conscientiousness", limit = None)
for post in subreddit_search:
    conscientiousness_user_names.append(post.author.name)

# Get unique only
conscientiousness_user_names = list(set(conscientiousness_user_names)) 

conscientiousness_df['User Name'] = conscientiousness_user_names
conscientiousness_df

,User Name
0,Proud_Echidna_4960
1,raisinindasun
2,Emma_Oakley
3,abbyrosalina
4,Luster-Star
...,...
160,lR5Yl
161,Cognitively_Absurd
162,_un1ty
163,killerlumberjack


# Create User Class

In [223]:
from enum import Enum
class Personality(Enum):
    EXTRAVERSION = 1,
    NEUROTICISM = 2,
    OPENNESS = 3,
    AGREEABLENESS = 4,
    CONSCIENTIOUSNESS = 5,

class User:
    def __init__(self, data_string, personality_type):
        self.data = data_string
        self.personality = personality_type
    def __str__(self):
        return str(self.personality) + "\n " +  str(self.data)


# Get User String Data

In [224]:
def getUserStringData(username):
    string_data = ""
    user = reddit.redditor(username)
    
    posts = list(user.submissions.new(limit = None))
    comments = list(user.comments.new(limit=None))
    if len(comments) < 5 and len(posts) < 5:
        return ""
    # filter posts
    for post in posts:
        string_data += post.title
        string_data += "\n" + post.selftext

    # filter comments
    for comment in comments:
        string_data += str(comment.body)
        string_data += "\n\n"
    string_data = string_data.replace("\n\n&#x200B;\n\n", " \n ")
    string_data = string_data.replace("&#x200B;", " \n")
    
    return string_data


In [225]:
from prawcore.exceptions import Forbidden

users = {}

# Add users from any group
def addUserDict(users_list, personality_type):
    for username in users_list:
        try:
            user_string_data = getUserStringData(username)
            if user_string_data == "":
                users_list.remove(username)
                continue
            user_data_object = User(user_string_data, personality_type)
            users[str(username)] = user_data_object
        except Forbidden:
            print("ERROR: USER/COMMENT NOT FOUND. RUN FOR YOU LIFE\n\n")
          
# test = ["Zestyclose-Tower3859", "Proud_Echidna_4960", "nationofcake", "Emma_Oakley", "Reghretti"]

# addUserDict(test, Personality.OPENNESS)
# for user in users:
#     print(f'_____________{user}________________')
#     print(users[user])

In [226]:
addUserDict(neuroticism_user_names, Personality.NEUROTICISM)
addUserDict(extraversion_user_names, Personality.EXTRAVERSION)
addUserDict(openness_user_names, Personality.OPENNESS)
addUserDict(agreeableness_user_names, Personality.AGREEABLENESS)
addUserDict(conscientiousness_user_names, Personality.CONSCIENTIOUSNESS)


ERROR: USER/COMMENT NOT FOUND. RUN FOR YOU LIFE




### Save all users and their data into a file for easy access 

In [ ]:
import json

with open("users.txt", "w") as file:
    json.dump(users, file)
print("Done writing dict into .txt file")

# Get Post From Username

In [ ]:
# Get username 
username = "lpetrich"

user = reddit.redditor(username)
submissions = user.submissions.new(limit = None)

for post in submissions:
    # Use post.title for title
    # Use post.selftext for text
    print(post.title)

Uncle Bernie translations: Tío Bernie, Amo Bernie, ...
Ranked-pairs algorithms?
Computational performance of vote-counting algorithms: How fast? How bulky?
Should US State Senates be abolished?
Latin American monarchs
Nineteenth-century European nation builders wanted monarchies
Republics to monarchies?
How Monarchies End
Why are the British royals such big celebrities?
Monarchical Republics?
Lindsay Holiday: Which European monarchies are next to fall?
Demographics: where do you live (more non-UK)?
Are monarchies more stable than republics?
What AOC might say to Kyrsten Sinema about Build Back Better
From exposure to lead: less conscientious, less agreeable, and more neurotic
US Presidents' Big Five Personalities
The Big Five vs. the Four Temperaments
The Big Five vs. the Enneagram
The Big Five vs. DISC
The Big Five vs. MBTI
Supertraits of the Big Five
The Evolution of the Big Five Factors
The Big Five factors in our brains
The Big Five, HEXACO, and the Dark Triad
Comparing the Big Fiv

# Average Posts per User

In [ ]:
from prawcore.exceptions import Forbidden
def getUniqueUsernames(all_usernames):
    unique_usernames = []

    for user in all_usernames:
        if user not in unique_usernames:
            unique_usernames.append(user)
    
    return unique_usernames

all_usernames =  neuroticism_user_names + extraversion_user_names + openness_user_names + agreeableness_user_names + conscientiousness_user_names
unique_usernames = getUniqueUsernames(all_usernames)

total_posts = 0

print(f'Total number of unique users: {len(unique_usernames)}')

for user in unique_usernames:
    # Get their reddit user profile
    reddit_user = reddit.redditor(user)
    submissions = reddit_user.submissions.new(limit = None)
    try:
        for post in submissions:
            total_posts += 1
    except Forbidden:
        print("Cant")
    
        

average_posts_per_user = total_posts/len(unique_usernames)

print(f'The average posts per user is {average_posts_per_user}')


Total number of unique users: 261
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant
Cant


KeyboardInterrupt: 